## 用Python写推荐系统

英文原文：
http://online.cambridgecoding.com/notebooks/eWReNYcAfB/implementing-your-own-recommender-systems-in-python-2

中文翻译：https://github.com/ictar/pythondocument/blob/master/Science%20and%20Data%20Analysis/%E5%9C%A8Python%E4%B8%AD%E5%AE%9E%E7%8E%B0%E4%BD%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E6%8E%A8%E8%8D%90%E7%B3%BB%E7%BB%9F.md

主要有两种推荐系统，基于内容的和协同过滤
Two most ubiquitous types of recommender systems are 
1. Content-Based and 
2. Collaborative Filtering (CF)

In [45]:
import numpy as np
import pandas as pd

数据将使用MovieLens数据集，它是在实现和测试推荐引擎时所使用的最常见的数据集之一。它包含来自于943个用户以及精选的1682部电影的100K个电影打分

In [46]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)

In [47]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items) )

Number of users = 943 | Number of movies = 1682


使用train_test_split将数据分为训练和测试两部分

In [48]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

CF可以分为
1. 基于内存的协同过滤和
2. 基于模型的协同过滤

CF can be divided into Memory-Based Collaborative Filtering and Model-Based Collaborative filtering. 

基于内存的协同过滤方法可以分为两个主要部分：用户-产品协同过滤和产品-产品协同过滤。

* 用户-产品协同过滤: “像你一样的人也喜欢……”
* 产品-产品协同过滤: “喜欢这个东西的人也喜欢……”

A user-item filtering will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. In contrast, item-item filtering will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations.

* Item-Item Collaborative Filtering: “Users who liked this item also liked …”
* User-Item Collaborative Filtering: “Users who are similar to you also liked …”

In [49]:
train_data.head()

,user_id,item_id,rating,timestamp
87715,869,127,5,884493279
99714,881,732,5,876538465
98918,70,94,3,884151014
15537,299,13,4,877877965
29034,13,79,3,882139746


In [50]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

创建的矩阵如下：
![](http://bos.nj.bpc.baidu.com/v1/agroup/fb1a8ac1e33b3b58716804b8b385e3a48796d1e8)

接着我们要计算相似性，相似性使用余弦来表示，我们可以将 train_data_matrix[1,:],train_data_matrix[2,:]每行看成是一个user feature的特征向量，然后计算两者之间的余弦

而 train_data_matrix[:,1] 列则是每个 item 的特征向量，我们通过pairwise_distances直接计算相似矩阵

In [51]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [52]:
user_similarity.shape

(943, 943)

In [53]:
item_similarity.shape

(1682, 1682)

In [14]:
# user_similarity[1,:] == user_similarity[:,1] # 对称矩阵

下面我们要做的事情是：
![](http://bos.nj.bpc.baidu.com/v1/agroup/f64581e5c5f932acce22be6cd2aca305fb29cd5e)
我们做的事情就是根据相似性和现有的知识来进行预测
此处基于用户相似性的预测有个注意的地方：
某些用户可能会倾向于对所有的电影，总是给予高或低评分。这些用户提供的评分的相对差比绝对评分值更重要。举个例子：假设，用户k对他最喜欢的电影打4星，而对所有其他的好电影打3星。现在假设另一个用户t对他/她喜欢的电影打5星，而对他/她感到无聊的电影打3星。
因此我们在计算用户对于产品的的打分的时候，取的是其平均值的差

我们先来看基于item的评分

In [58]:
# 第一个用户的前10个对于产品的rate
train_data_matrix[0,:10] 

array([ 0.,  0.,  4., ...,  0.,  0.,  0.])

In [65]:
# 我们要预测产品0的rate
# 我们得到产品0和其他产品的相关性
item_similarity[0,:]

array([ 0.        ,  0.66040109,  0.7557665 , ...,  1.        ,
        0.94627847,  1.        ])

In [66]:
# 下一步就是计算了
train_data_matrix[0,:].dot(item_similarity[0,:].T) / item_similarity[0,:].sum()

0.36884840159267351

In [ ]:
# 下面我们来基于user进行预测
# 第一个用户的前10个对于产品的rate
train_data_matrix[0,:10] 

In [69]:
# 我们要预测产品0的rate
# 我们得到用户0和其他用户的相关性
user_similarity[0,:10]

array([ 0.        ,  0.84903016,  0.95543051,  0.95178585,  0.72141644,
        0.62258281,  0.67164805,  0.80065886,  0.93744475,  0.75892725])

In [75]:
# 其他用户对于产品0的评价是 train_data_matrix[:,[0]]
a = train_data_matrix[:,0].mean() # 计算第一列的平均值
train_data_matrix.mean(axis=0)[0] == a # axis = 0 表示按列求平均值

True

In [76]:
a = train_data_matrix[0,:].mean() # 计算第1行的平均值
train_data_matrix.mean(axis=1)[0] == a # axis = 1 表示按行求平均值

True

In [83]:
# train_data_matrix.mean(axis=1)[:,None] # 转换为列
# train_data_matrix[0,:].mean()  为用户0对产品打分的平均值
train_data_matrix[0,:].mean() + ( train_data_matrix[:,0] - train_data_matrix.mean(axis=1) ).dot(user_similarity[:,[0]]) / user_similarity[:,[0]].sum()

array([ 1.63288052])

In [60]:
#train_data_matrix[:10,:10]

array([[ 0.,  0.,  4.,  3.,  0.,  5.,  4.,  1.,  5.,  3.],
       [ 4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0.,  2.,  4.,  4.,  0.],
       [ 0.,  0.,  0.,  5.,  0.,  0.,  5.,  0.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  0.]])

In [85]:
def predict(ratings, similarity, type='user'):
        if type == 'user':
            mean_user_rating = ratings.mean(axis=1)
            #You use np.newaxis so that mean_user_rating has same format as ratings
            ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
            pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        elif type == 'item':
            pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
        return pred

In [86]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

有了评估值，下面我们用做的就是衡量这些评估值的准确性了，使用下面的公式来评价
![](https://camo.githubusercontent.com/da51be69e6899ef7a52095d1e9f5b6479db51497/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f6769662e6c617465783f524d53452673706163653b3d2535437371727425374225354366726163253742312537442537424e2537442673706163653b25354373756d2673706163653b28785f692673706163653b2d253543686174253742785f692537442925354532253744)

In [84]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [88]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.123821413757305
Item-based CF RMSE: 3.449900493815728


内存的CF很简单，但是问题有：
1. 数据规模变大的时，不可能实时计算
2. 冷启动问题，刚开始时，没有用户数据，怎么推荐